In [4]:
import pickle
import numpy as np

In [5]:
with open("fhn_dirichlet_obs.p", "rb") as f:
    d = pickle.load(f)
d.keys()

dict_keys(['Xtrue', 'Ytrain', 'Yvalid'])

In [6]:
x = d['Xtrue']
y_train = d['Ytrain']
y_test = d['Yvalid']
x_train = x[:len(y_train)]
x_test = x[len(y_train):]
print(x.shape, y_train.shape, y_test.shape)

(240, 200, 2) (200, 200, 11) (40, 200, 11)


In [7]:
y_train_day = np.tile(np.arange(200)[np.newaxis, :], (len(y_train), 1))
y_train_day = y_train_day[:, :, np.newaxis]
y_train_new = np.concatenate((y_train_day, y_train), axis=-1)
y_test_day = np.tile(np.arange(200)[np.newaxis, :], (len(y_test), 1))
y_test_day = y_test_day[:, :, np.newaxis]
y_test_new = np.concatenate((y_test_day, y_test), axis=-1)
print(y_train_new.shape, y_test_new.shape)
print(y_train_new[0, :10, :])

(200, 200, 12) (40, 200, 12)
[[0.00000000e+00 4.85006007e-04 3.94206805e-03 4.03809587e-01
  4.64151866e-02 9.92533746e-02 2.09051320e-01 1.31066515e-01
  1.98086308e-02 8.08765937e-03 6.89980457e-02 9.08260710e-03]
 [1.00000000e+00 7.96099640e-05 2.72677746e-03 3.81801545e-01
  8.86917020e-03 3.36439118e-03 1.09892458e-01 3.43266689e-01
  5.31109489e-02 1.94761649e-04 4.11680646e-02 5.55255849e-02]
 [2.00000000e+00 2.16449375e-02 3.81088935e-02 1.33266312e-01
  6.76578112e-02 1.81963937e-01 8.32166717e-02 2.27035453e-01
  1.16089252e-05 1.23819588e-01 2.55146447e-02 9.77601418e-02]
 [3.00000000e+00 1.76730916e-09 1.36653429e-02 1.33656055e-01
  4.80006945e-03 1.05323789e-01 2.57071820e-02 2.67050669e-01
  1.88218906e-02 9.23826337e-02 3.09314185e-03 3.35499224e-01]
 [4.00000000e+00 1.01767752e-08 8.11034690e-02 3.32498266e-01
  8.29847202e-09 3.55059881e-02 1.51638242e-01 8.08513369e-02
  3.88350149e-09 1.18237765e-01 1.69293895e-01 3.08710152e-02]
 [5.00000000e+00 1.67695949e-03 1.15

In [8]:
# 20 % missing obs
T = y_train_new.shape[1]
for obs_percentage in [0.8, 0.6, 0.5, 0.4]:
    y_train_tmp = []
    y_test_tmp = []
    for y in y_train_new:
        obsed_days = np.random.choice(np.arange(T), int(T * obs_percentage), replace=False)
        obsed_days = np.sort(obsed_days)
        y_train_tmp.append(y[obsed_days])
    for y in y_test_new:
        obsed_days = np.random.choice(np.arange(T), int(T * obs_percentage), replace=False)
        obsed_days = np.sort(obsed_days)
        y_test_tmp.append(y[obsed_days])
    y_train_tmp = np.stack(y_train_tmp)
    y_test_tmp = np.stack(y_test_tmp)
    print(y_train_tmp.shape, y_test_tmp.shape)
    days = np.arange(T)[np.newaxis, :, np.newaxis]
    v_train = np.concatenate([np.tile(days, (len(y_train_new), 1, 1)), np.zeros((len(y_train_new), T, 1))], axis=-1)
    v_test = np.concatenate([np.tile(days, (len(y_test_new), 1, 1)), np.zeros((len(y_test_new), T, 1))], axis=-1)
    with open("fhn_{}_dirichlet_obs.p".format(obs_percentage), "wb") as f:
        pickle.dump({'Ytrain': y_train_tmp, 'Ytest': y_test_tmp, 'Vtrain': v_train, 'Vtest': v_test}, f)

(200, 160, 12) (40, 160, 12)
(200, 120, 12) (40, 120, 12)
(200, 100, 12) (40, 100, 12)
(200, 80, 12) (40, 80, 12)


In [9]:
print(v_train[0, :10])

[[0. 0.]
 [1. 0.]
 [2. 0.]
 [3. 0.]
 [4. 0.]
 [5. 0.]
 [6. 0.]
 [7. 0.]
 [8. 0.]
 [9. 0.]]
